#Корпус решений Верховного суда Нидерландов по гражданским делам за 2025 год

Корпус решений Верховного суда Нидерландов (Hoge Raad der Nederlanden) нужен для выполнения чат-ботом своей второй функции - выдачи по запросу пользователя текста судебного решения определенного размера и уровня лексического разнообразия со списком ключевых слов.

#1. Загрузка библиотек

In [ ]:
!pip install stanza
import stanza
stanza.download('nl')
nlp = stanza.Pipeline(lang = 'nl')

!pip install pandas
import pandas as pd

from collections import Counter

import os

import re

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('dutch')
print(stop_words)

In [ ]:
from collections import Counter

#2. Определение функций

##2.1. Первичная обработка текста - initial_clean_the_text
Функция разделяет "слипшиеся" буквы и цифры и корректирует отображение в тексте нумерации.


Она будет использоваться для чистки текста перед выдачей его пользователю, а также для первичной обработки текста перед направлением его в Stanza и подсчетом TTR и TF-IDF.

In [ ]:
def initial_clean_the_text(doc):
  doc_clean = re.sub(r'(?<=\d+)(?=[A-Z])', '. ', doc_clean)
  doc_clean = re.sub(r'(?<=[A-Za-z])(?=\d+)', ' ', doc_clean)
  doc_clean = re.sub(r'(?<=\d\.\d)\s', '. ', doc_clean)

  return doc_clean

##2.2. Финальная обработка текста - final_clean_the_text
Функция применяется для обработки текста перед направлением его в Stanza и подсчетом TTR и TF-IDF. В ней важно было соблюсти баланс между чисткой текста и сохранением необходимого материала для корректной работы Stanza.

Функция выполняет три основных задачи:  
(1) заменяет обезличенные фрагменты текстов судебных решений на заглушку-NER, чтобы в дальшейшем их удалить, как это будет сделано с NER в других текстах;  
(2) убирает лишний шум (коды судебных решений, нумерацию);  
(3) вносит в текст небольшие правки, чтобы Stanza корректно проставила частеречную и синтаксическую разметку.

Для решения задачи (1) дополнительно были собраны данные относительно того, какие фрагменты текстов судебных решений суд обычно помещает в квадратные скобки - только обезличенные данные об участниках процесса или же ещё какие-либо элементы текста. Выяснилось, что в квадратные скобки суд также помещает ошибки, опечатки, названия законов, указания на объекты недвижимости. В связи с этим чистка текста от обезличенных данных была проведена адресно, только в отношении тех фрагментов, которые можно отнести к NER (подробнее см. раздел 3 тетрадки).

Задача (3) появилась после того, как в работе Stanza были обнаружены неточности, которые, однако, можно устранить. Например:  
- Stanza хорошо распознает NER, когда инициала всего 2, когда же их 3, она начинает неверно проставлять разметку, поэтому там, где в имени человека 3 инициала, его имя заменяется на заглушку-NER;  
- слова, написанные полностью с заглавных букв, Stanza неверно лемматизирует, вплоть до того, что в качестве леммы указываются несуществующие слова, в связи с этим названия участников процесса (истец, ответчик и пр.) были приведены полностью к нижнему регистру, а в остальных словах, состоящих только из заглавных букв, заглавной была оставлена только первая буква, чтобы Stanza могла распознать их как потенциальный NER.


In [13]:
def final_clean_the_text(doc_clean):
  final_doc_clean = re.sub(r'ECLI:NL:\w+:\d+:\d+', ' ', doc_clean)
  try:
    xx = re.findall(r'(?<=\nadvocaat:)[\w\s\.&-]+?(?=[\.,]\n)', final_doc_clean)
    if len(xx) > 0:
      for i in xx:
        final_doc_clean = re.sub(f'{i}', ' Marijke ', final_doc_clean)
  except:
   pass

  try:
    xx = re.findall(r'(?<=\nIn de zaak van\n\n)[A-Z][\w\s\.&-]+?(?=[\.,]\n)', final_doc_clean)
    if len(xx) > 0:
      for i in xx:
        final_doc_clean = re.sub(f'{i}', ' Anneke ', final_doc_clean)
  except:
    pass

  try:
    xx = re.findall(r'(?<=\nhierna:\s)[A-Z][\w\s\.&-]+?(?=[\.,]\n)', final_doc_clean)
    if len(xx) > 0:
      for i in xx:
        final_doc_clean = re.sub(f'{i}', ' Tom ', final_doc_clean)
  except:
    pass

  try:
    xx = re.findall(r'(?<=\ntegen\n\n)[A-Z][\w\s\.&-]+?(?=[\.,]\n)', final_doc_clean)
    if len(xx) > 0:
      for i in xx:
        final_doc_clean = re.sub(f'{i}', ' Gert ', final_doc_clean)
  except:
    pass

  try:
    xx = re.findall(r'(?<=\nIn de zaak van\n\n\d\.\s)[A-Z][\w\s\.&-]+?(?=[\.,]\n)', final_doc_clean)
    if len(xx) > 0:
      for i in xx:
        final_doc_clean = re.sub(f'{i}', ' Gert ', final_doc_clean)
  except:
    pass

  try:
    xx = re.findall(r'(?<=\n\n\d\.\s)[A-Z][\w\s\.&]+?(?=[\.,]\n\n)', final_doc_clean)
    if len(xx) > 0:
      for i in xx:
        final_doc_clean = re.sub(f'{i}', ' Gert ', final_doc_clean)
  except:
    pass

  try:
    xx = re.findall(r'(?<=\n\nhierna gezamenlijk:\s)[A-Z][\w\s\.&]+?(?=[\.,]\n\n)', final_doc_clean)
    if len(xx) > 0:
      for i in xx:
        final_doc_clean = re.sub(f'{i}', ' Gert en Tom ', final_doc_clean)
  except:
    pass

  final_doc_clean = re.sub(r'([A-Z]\.\s)?[A-Z]\.\s[A-Z]\.\s[A-Z]\.\s(\w+\s)?[A-Z][a-z]+', ' Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\b\d\.', ' ', final_doc_clean)
  final_doc_clean = re.sub(r'(?<=[a-z])/(?=[a-z])', ' of ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[A-Z]\]|\[[A-Z][A-Z]\]', ' Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[A-Z][A-Z]\s[a-z]\.[a-z]\.\]', ' Marijke en Anneke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[lL]eeftijd\]', ' 5 ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[mM]aand, jaar\]', ' april 1931 ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[eE]iser\]|\[[eE]iseres\]|\[[eE]iser \([\w\s/]+\)\]|\[[eE]iser \d+\]|\[[eE]iseres \d+\]|\[[vV]erzoek(er|ster) \d+\]|\[[vV]erzoek(er|ster)\]|\[[Aa]ppellant\]|\[[Cc]hauffeur\]', ' Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[eE]isers\]|\[[eE]iseressen\]|\[[eE]isers \([\w\s/]+\)\]|\[[vV]erzoek(ers|sters)([\s\w/]+)?\]', ' Marijke en Anneke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[vV]erweer(der|ster)\]|\[[vV]erweer(der|ster) \([\w\s/]+\)\]|\[[vV]erweer(der|ster) \d+([a-z])?\]|\[[bB]elanghebbende\]|\[[bB]elanghebbende \d+\]|\[[Bb]etrokkene\]|\[[Bb]etrokkene \d+\]', ' Anneke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[vV]erweer(der|ster)s([\s\w/]+)?\]|\[[vV]erweer(der|ster)s \([\w\s/]+\)\]|\[[bB]elanghebbenden\]|\[[bB]etrokkenen ([\s\w/]+)?\]', ' Anneke en Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[([Dd]e )?[Ww]erkne(emster|mer)\]|\[[Bb]roer( \d+)?\]|/[een derde/]', ' Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[([dD]e )?(zus|dochter|zoon|broer|zakenpartner|halfbroer|moeder|statutair bestuurder|bewindvoerder|koper|benadeeelde|oma|buurman|zorgaanbieder|minderjarige|schoonmaakster|vereffenaar|ingeleende|leidinggevende|agente|juridische dienstverlener|bewoner|verhuurder|man|Fysiotherapeut|vrouw|echtgenote|vader|derde|advocaat|commissaris|bestuurder|familie)( [\dA-Z]+)?\]', ' Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[([dD]e |[bB]edoelde |[Bb]epaalde |[Dd]eze )?(ouders|eisers)\]', ' Anneke en Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[([dD]e zus en de broer)|([dD]e broer en de zus)|([dD]e vader en de moeder)|([dD]e moeder en de vader)|([kK]inderen van overleden huurster)|([dD]e erven)\]', ' Anneke en Marijke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[lL]and\]', ' Nederland ', final_doc_clean)
  final_doc_clean = re.sub(r'\[([Bb]eheer|[Bb]edrijf|[vV]ennootschap|de voormalige vennoot)( \d+)?\]', ' AAB ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[Nn]aam( klant)?\]', ' Anneke ', final_doc_clean)
  final_doc_clean = re.sub(r'\[([vV]estigings|[wW]oon|[Vv]erblijf|[Gg]eboorte)plaats\]', ' Amsterdam ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[pP]laats(naam)?\]', ' Rotterdam ', final_doc_clean)
  final_doc_clean = re.sub(r'\[[ixv]+\]', ' ', final_doc_clean)
  # final_doc_clean = re.sub(r'\bMCA\b', 'master clearing agreements', final_doc_clean)
  # final_doc_clean = re.sub(r'\bGCFA\b', 'group credit facility agreement', final_doc_clean)
  # final_doc_clean = re.sub(r'\bGUA\b', 'give up agreement', final_doc_clean)
  # final_doc_clean = re.sub(r'\bGCS\b', 'garanteed settlement agreement', final_doc_clean)
  final_doc_clean = re.sub(r'\]|\[', ' ', final_doc_clean)
  final_doc_clean = re.sub(r'\([ixv]+\)', ' ', final_doc_clean)
  final_doc_clean = final_doc_clean.replace('Nummer', 'nummer').replace('Datum', 'datum').replace('EISERESSEN', 'eiseressen').replace('EISERES', 'eiseres').replace('EISER', 'eiser').replace('VERWEERSTERS', 'verweersters').replace('VERWEERDERS', 'verweerders').replace('VERWEERSTER', 'verweerster').replace('VERWEERDER', 'verweerder')

  naive_tokens = []
  for i in final_doc_clean.split():
      if i.isupper():
        naive_tokens.append(f'{i[0]}{i[1:].lower()}')
      else:
        naive_tokens.append(i)

  final_doc_clean = ' '.join(naive_tokens)

  return final_doc_clean

##2.3. Обработка текста в Stanza и получение первичного дата-фрейма - get_data_frame_common
Функция собирает материал для дальнейшего подсчета TTR текста судебного решения и первичный материал для подсчета TF-IDF.

Пошагово внутри функции происходит следующее:  
(1) текст обрабатывается в Stanza;  
(2) функция итерируется по результатам работы Stanza: отдельно по частеречной разметке (в sentence.words), отдельно по NER (в sentence.tokens), и формирует 2 дата-фрейма, которые затем соединяются в один по sentenсe id и word id; sentenсe id формируется с помощью count, поскольку в Stanza нет нумерации предложений;  
(3) дата-фрейм несколько раз фильтруется, чтобы убрать из него:  
- служебные части речи, NER, символы, числа;
- стоп-слова (взяты из библиотеки NLTK, дополнительного списка стоп-слов, найденного в интернете - [здесь](https://github.com/stopwords-iso/stopwords-nl), также добавила отдельные стоп-слова из текстов судебных решений, которые выявила при анализе текстов);  
- нижние подчеркивания, которые обозначают в Stanza отделяемые части слов (непринципиальны для сбора корпуса);  
- лишний шум (слова менее 3 символов, которые обычно представляют собой сокращения и нераспознанные Stanza части NER);  

(4) дата-фрейм обрезается по слову "uitspreken", после которого в текстах судебных решений идут сноски со ссылками на структурные единицы нормативных актов и иные судебные решения (непринципиальны для сбора корпуса).


In [10]:
def get_data_frame_common(cleaned_document):
  stanza_clean = nlp(cleaned_document)
  for_data_frame_pos = []
  count = 0
  for sentence in stanza_clean.sentences:
      for word in sentence.words:
          for_data_frame_pos.append({'sentence_id': count, 'word_id': word.id, 'word': word.text,
                                'lemma': word.lemma, 'upos': word.upos, 'xpos': word.xpos,
                                'head': word.head, 'deprel': word.deprel})
      count = count + 1
  df_pos = pd.DataFrame(for_data_frame_pos)

  for_data_frame_ner = []
  count = 0
  for sentence in stanza_clean.sentences:
      for token in sentence.tokens:
          for_data_frame_ner.append({'sentence_id': count, 'word_id': token.id[0], 'word': token.text, 'ner': token.ner})
      count = count + 1
  df_ner = pd.DataFrame(for_data_frame_ner)
  df_ner_selected = df_ner[['sentence_id', 'word_id', 'ner']]

  merged_df_pos_ner = pd.merge(df_pos, df_ner_selected, on=['sentence_id', 'word_id'], how='outer')

  del_pos = ['NUM', 'X', 'SYM', 'PUNCT', 'DET', 'AUX', 'PRON', 'CCONJ', 'INTJ', 'ADP', 'SCONJ']
  merged_df_pos_ner_filtered = merged_df_pos_ner.loc[~merged_df_pos_ner['upos'].isin(del_pos)]

  del_ner = ['B-PER', 'I-PER', 'E-PER', 'S-PER', 'B-ORG', 'I-ORG', 'E-ORG', 'S-ORG', 'B-LOC', 'I-LOC', 'E-LOC', 'S-LOC', 'I-MISC', 'B-MISC', 'E-MISC', 'S-MISC']
  merged_df_pos_ner_filtered_2 = merged_df_pos_ner_filtered.loc[~merged_df_pos_ner_filtered['ner'].isin(del_ner)].copy()

  merged_df_pos_ner_filtered_2['lemma'] = merged_df_pos_ner_filtered_2['lemma'].apply(lambda x: re.sub('_', '', x))

  merged_df_pos_ner_filtered_3 = merged_df_pos_ner_filtered_2.loc[merged_df_pos_ner_filtered_2['lemma'].str.len() > 2]

  merged_df_pos_ner_filtered_4 = merged_df_pos_ner_filtered_3[~merged_df_pos_ner_filtered_3['word'].str.contains(r'\d+', regex=True)].copy()

  merged_df_pos_ner_filtered_4['lemma'] = merged_df_pos_ner_filtered_4['lemma'].str.lower()

  del_word = stop_words + ['zult', 'betreffende', 'vallen', 'rechter', 'urw', 'vier', 'Ehg-nl', 'aldaar', 'miss', 'aangezien', 'voort', 'dat', 'duizend', 'vanwege', 'niets', 'hiervan', 'al', 'blijkbaar', 'altijd', 'jij', 'terzijde', 'vooruit', 'omlaag', 'meestal', 'jouwe', 'wederom', 'meest', 'zowat', 'doen', 'zijn', 'ik', 'neem', 'tenzij', 'nee', 'hierbeneden', 'daarvoor', 'hierom', 'niks', 'ons', 'wanneer', 'mocht', 'tien', 'even', 'zulk', 'dertig', 'tot', 'voorbij', 'sbk', 'nadat', 'gekund', 'overal', 'ov', 'niemand', 'alias', 'zelfs', 'gedurende', 'kan', 'zei', 'jouw', 'kunnen', 'samen', 'hoe', 'rond', 'mijn', 'werden', 'toch', 'hare', 'zouden', 'meer', 'noch', 'etc', 'was', 'zal', 'wijzelf', 'achte', 'anderszins', 'beiden', 'niet', 'lijkt', 'sinds', 'wzd', 'weinig', 'mrs', 'is', 'er', 'waar', 'waarboven', 'doorgaand', 'nogal', 'wel', 'beide', 'llmc', 'zijzelf', 'anders', 'daarvanlangs', 'de', 'uw', 'nu', 'daar', 'missen', 'aan', 'maken', 'waarbij', 'ander', 'nergens', 'honderd', 'krachtens', 'waarvan', 'toe', 'eerlang', 'lifex', 'aangaande', 'na', 'voorts', 'wat', 'word', 'iedereen', 'daarnet', 'inmiddels', 'je', 'hoewel', 'heb', 'ja', 'af', 'misschien', 'nemen', 'en', 'lgr', 'wie', 'enig', 'moet', 'maar', 'verscheidene', 'iets', 'alles', 'kun', 'nooit', 'zodat', 'bovenvermeld', 'daarheen', 'aldus', 'jullie', 'doe', 'zonder', 'kunt', 'hebben', 'hunne', 'vooralsnog', 'bijna', 'ooit', 'bv', 'zichzelf', 'desbetreffende', 'zeven', 'tijdens', 'waarschijnlijk', 'eer', 'eerst', 'slechts', 'omdat', 'jegens', 'daarna', 'laatst', 'andere', 'kon', 'bijv', 'echter', 'wil', 'bns', 'erdoor', 'deden', 'gewoon', 'eigenlijk', 'als', 'liever', 'derde', 'onder', 'steeds', 'overige', 'vol', 'daarom', 'drie', 'eens', 'het', 'zelf', 'wegens', 'tweede', 'vooraf', 'zulks', 'klaar', 'juist', 'zojuist', 'inderdaad', 'geleden', 'per', 'mogen', 'enigszins', 'deed', 'waarover', 'rondom', 'gewoonweg', 'negen', 'gauw', 'vijftig', 'ongeveer', 'heel', 'konden', 'vanuit', 'boven', 'haarzelf', 'maakt', 'want', 'zijne', 'hun', 'behalve', 'gelijk', 'moest', 'door', 'verschillende', 'ondertussen', 'alleen', 'totdat', 'elk', 'voor', 'nv', 'werd', 'onzeker', 'toenmaals', 'hierna', 'om', 'zoals', 'jou', 'te', 'zullen', 'willen', 'maakte', 'waren', 'recent', 'weldra', 'nochtans', 'binnenin', 'ge', 'voorop', 'beneden', 'dus', 'elke', 'vandaan', 'we', 'immers', 'die', 'wordt', 'waarnaar', 'ieder', 'hij', 'verder', 'dan', 'bent', 'toenmalig', 'tegenover', 'waarvoor', 'vaakwat', 'bepaald', 'ergens', 'alle', 'eerder', 'overeind', 'zeker', 'clni', 'enige', 'erg', 'pmt', 'hen', 'waarom', 'overigens', 'vanaf', 'onderhavige', 'voorheen', 'daardoor', 'vrij', 'redelijk', 'vervolgens', 'bovengenoemd', 'alhoewel', 'zij', 'achter', 'iedere', 'hier', 'altoos', 'vorenstaande', 'eigen', 'zodra', 'over', 'enkel', 'lijken', 'iemand', 'van', 'bovendien', 'haar', 'welk', 'ben', 'mag', 'wezen', 'bovenal', 'mr', 'maakten', 'hemzelf', 'sindsdien', 'daarop', 'zulke', 'zover', 'omtrent', 'via', 'hem', 'geweest', 'worden', 'hierboven', 'bovenstaand', 'veel', 'behoudens', 'uitgezonderd', 'überhaupt', 'tiende', 'hadden', 'zo', 'allebei', 'wij', 'twee', 'voordien', 'werder', 'expl', 'ondanks', 'voordat', 'hijzelf', 'hetzelfde', 'bij', 'hierin', 'genoeg', 'derhalve', 'zes', 'vroeg', 'op', 'doet', 'etcetera', 'eerste', 'mits', 'blijken', 'mede', 'thans', 'inzake', 'precies', 'alt', 'tja', 'welke', 'wiens', 'zich', 'vijfde', 'volgens', 'terwijl', 'geen', 'mijnent', 'mochten', 'pas', 'later', 'naar', 'veertig', 'waarop', 'eveneens', 'weg', 'hedden', 'daaruit', 'omstreeks', 'veeleer', 'vijf', 'zelfde', 'hebt', 'evenwel', 'waaruit', 'moeten', 'opzij', 'mijner', 'hiervoor', 'eerdat', 'beetje', 'binnen', 'dikwijls', 'ze', 'u', 'had', 'moesten', 'voordezen', 'gemogen', 'volgend', 'gemoeten', 'net', 'opnieuw', 'onze', 'daarin', 'waaraan', 'omver', 'dhr', 'jezelf', 'me', 'omhoog', 'wier', 'vierde', 'sommige', 'deze', 'met', 'mezelf', 'men', 'ikzelf', 'weer', 'wilden', 'tussen', 'reeds', 'vaak', 'sedert', 'tamelijk', 'een', 'toen', 'minder', 'jijzelf', 'maak', 'namelijk', 'nog', 'cv', 'dit', 'der', 'heeft', 'gehad', 'ofschoon', 'mijzelf', 'dergelijke', 'spoedig', 'in', 'ook', 'tegen', 'onszelf', 'mevr', 'paar', 'zou', 'nam', 'of', 'buiten', 'achterna', 'vooral', 'intussen', 'alsnog', 'óók', 'mw', 'doch', 'uit', 'mij']
  merged_df_pos_ner_filtered_5 = merged_df_pos_ner_filtered_4.loc[~merged_df_pos_ner_filtered_4['word'].isin(del_word)]
  merged_df_pos_ner_filtered_6 = merged_df_pos_ner_filtered_5.loc[~merged_df_pos_ner_filtered_5['lemma'].isin(del_word)]

  to_trim = merged_df_pos_ner_filtered_6.index[merged_df_pos_ner_filtered_6['lemma'] == 'uitspreken']
  list_to_trim = to_trim.tolist()
  ind = list_to_trim[-1]

  df_trimmed = merged_df_pos_ner_filtered_6.loc[:ind]

  return df_trimmed

##2.4. Получение дата-фрейма для выявления ключевых слов - get_data_frame_key_words
Функция чистит первичный дата-фрейм, собранный в предыдущей функции, для его дальнейшего использования при подсчете TF-IDF.

Из первичного дата-фрейма удаляются типичные для юридического языка слова, такие как "истец", "ответчик", "жалоба", "местонахождение" и другие. Эти слова не вычищаются с помощью TF-IDF, потому что присутствуют не в каждом тексте судебного решения, а если присутствуют, то в разных объемах - так, что иногда TF-IDF начинает их выделять в качестве ключевых слов, в то время как по этим словам невозможно составить представление о сути судебного разбирательства.

Слова отобраны частично на основе данных из интернета о типичной юридической терминологии, частично вручную - по итогам первой апробации функции на корпусе текстов судебных решений, частично с учетом моего опыта работы с юридическиими текстами. Таким образом, в списке юридических стоп-слов могут быть собраны не все слова, которые могли бы туда попасть при отборе слов на основе обработки большого корпуса правовых текстов (такая задача не вошла в задачи проекта).



In [ ]:
def get_data_frame_key_words(df_common):
  del_words = ['lisse', 'bodemprocedure', 'ehg', 'eiseres', 'whw', 'kantoorhouden', 'genaamd', 'hoog', 'eindarrest', 'lid', 'maart', 'griffier', 'cashdepot', 'hypotheekviseur', 'raadvrouw', 'verzoekster', 'rechtsvordering', 'get', 'verzetarrest', 'eiseressen', 'reageren', 'Procesverloop', 'vermelding', 'hoogte', 'verstekarrest', 'jas', 'hoeven', 'moeder', 'spreken', 'leggen', 'wethouder', 'rzv', 'kilometer', 'kosten', 'zetelen', 'abc', 'vof', 'belanghebbende', 'raad', 'verwerping', 'conclusie', 'vestigingplaats', 'wamrichtlijn', 'betrokkene', 'tijdelijk', 'verzoeksters', 'rechtbank', 'vordering', 'partij', 'governance', 'lwb', 'curator', 'organizatie', 'nummer', 'verweersters', 'ondertekenen', 'zaak', 'heden', 'president', 'caot', 'titel', 'kvz', 'klachtprocedure', 'beslissing', 'weesp', 'eugs', 'bepaling', 'juridisch', 'wvggz', 'klachtbeoordelen', 'doen', 'voorwaardelijk', 'zus', 'verweerder', 'uitspreken', 'aangifte', 'doel', 'verzoeker', '290bedrijfsruimte', 'verweer', 'raadsheer', 'commissaris', 'stul', 'beoordeling', 'incidenteel', 'gezamenlijk', 'aanhef', 'toezegging', 'indienen', 'procureur', 'statuut', 'februari', 'eindvonnis', 'herstelarrest', 'geven', 'verweerschrift', 'verbaal', 'ems', 'caobepaling', 'verschijnen', 'lovers', 'cassatie', 'bns', 'divi', 'rol', 'gerechtshof', 'rit', 'naam', 'eye', 'aanleg', 'eindbeslissing', 'voegen', 'zitiing', 'instellen', 'rechtercommissaris', 'uitgangspunt', 'advocaat', 'arrondissement', 'oktober', 'principaal', 'azm', 'Kayooms', 'tussenvonnis', 'vader', 'bericht', 'kubus', 'geding', 'wetboek', 'veste', 'gerecht', 'wet', 'feit', 'mei', 'hof', 'akte', 'verzoekschrift', 'vennootschap', 'pot', 'vennot', 'cassatieberoep', 'bruto', 'dusver', 'achtereenvolgend', 'tan', 'verweerders', 'november', 'instantie', 'eiser', 'tweede', 'wanbeleid', 'eisers', 'categorie', 'tegenbewijs', 'cogsa', 'pkn', 'schriftelijk', 'rechtvaardiging', 'hoedanigheid', 'verz', 'urw', 'ter', 'zijde', 'meter', 'september', 'artikel', 'vestigen', 'kost', 'prejudiciële', 'kantonzaak', 'officier', 'beschikking', 'oordeel', 'gedingstuk', 'bewijs', 'toelichten', 'cao', 'termijn', 'openbaar', 'rvverzoek', 'BWS', 'evrm', 'verwijzen', 'wzd', 'ue', 'megaapbbennootschap', 'datum', 'incidentelen', 'verlenen', 'uu', 'voorhand', 'verweerster', 'wijziging', 'eindbeschikking', 'uitkomst', 'voetnoot', 'sagel', 'college', 'verzoekers', 'recht', 'herstelvonnis', 'augustus', 'verloop', 'fase', 'cnb', 'rechthebbende', 'verstek', 'juni', 'catchers', 'vandaag', 'wam', 'tussentijds', 'wan', 'kort', 'maand', 'justitie', 'mhr', 'extra', 'kantonrechter', 'december', 'belang', 'voorzitter', 'oud', 'midden', 'vraag', 'nai', 'verzoek', 'regel', 'april', 'rechterlijk', 'wetgeving', 'maken', 'zaaknummer', 'klacht', 'gemeente', 'middel', 'moving', 'brief', 'aua', 'januari', 'rechtvraag', 'vonnis', 'rele', 'uitstrekken', 'miljard', 'juli', 'volgen', 'man', 'advocaat-generaal', 'broer', 'belanghebben', 'deskundig', 'antwoord', 'raadsman', 'feitelijk', 'heusden', 'incident', 'adres', 'vroeg', 'staatssecretaris', 'op', 'ondernemingskamer', 'beroep', 'richtlijn', 'maatschap', 'dragen', 'principale', 'maatregel', 'jaar', 'arbeidsomstandighedenwetenwetenwetenwetenweten', 'karakter', 'de instelling', 'vicepresident', 'dws', 'uitspraak', 'mail', 'vierkant', 'leiden', 'zien', 'stil', 'sxm', 'vrouw', 'proces', 'dag', 'wonen', 'komen', 'arrest', 'procesverloop']
  df_for_key_words = df_common.loc[~df_common['word'].isin(del_words)]
  df_for_key_words_2 = df_for_key_words.loc[~df_for_key_words['lemma'].isin(del_words)]

  return df_for_key_words_2

##2.5. Подсчет TTR - get_ttr
Функция считает TTR (type-token ratio) для оценки лексического разнообразия текста судебного решения.

Поскольку тексты судебных решений в корпусе разного размера, используется стандартизированный TTR с окном в 200 лемм. При этом для текстов, в которых меньше 200 лемм, TTR хоть и считается, но в дальнейшем не используется в чат-боте, чтобы не исказить данные. Кроме того, для маленьких текстов судебных решений TTR непоказателен. Небольшие тексты судебных решений типовые, в них содержится лишь отсылка на решения нижестоящих судов и указание на то, что Верховный суд поддерживает эти решения.

Значение в 200 лемм выбрано в качестве порогового по итогам соотношения наивно подсчитанной длины текстов судебных решений и количества содержащихся в них лемм. Текст судебного решения перестает быть типовым после примерно 600 наивных токенов, что соответствует числу в 200 лемм (подробнее см. раздел 3 тетрадки).

Также учтено, что окно в 200 лемм по-разному укладывается в тексты судебных решений: где-то последнее окно близко к 200, а где-то намного меньше, что может приводить к безосновательному завышению TTR. Чтобы смягчить этот эффект, последнее окно, в зависимости от его размера, либо считается отдельно (если оно ближе к 200, чем к 0), либо присоединяется к предыдущему (если оно ближе к 0, чем к 200).

In [4]:
def get_ttr(df):
  if df.shape[0] <= 200:
    num_types = df['lemma'].nunique()
    num_tokens = df.shape[0]
    ttr = round((num_types / num_tokens)*100, 3)

  elif df.shape[0] > 200:
    lemma_list = df['lemma'].tolist()
    number_of_lemmas = len(lemma_list)

    ttr_200_all = []
    n = 0
    k = 200
    count = 0

    while count <= (number_of_lemmas/200):
      if k <= number_of_lemmas:
        types = len(set(lemma_list[n:k]))
        ttr_200 = round((types / 200)*100, 3)
        ttr_200_all.append(ttr_200)
        n = n + 201
        k = k + 201
      elif (k > number_of_lemmas) and ((number_of_lemmas/200 % 1) >= 0.5):
        types = len(set(lemma_list[n:]))
        ttr_200 = round((types / 200)*100, 3)
        ttr_200_all.append(ttr_200)
      elif (k > number_of_lemmas) and ((number_of_lemmas/200 % 1) < 0.5):
        del ttr_200_all[-1]
        n = n - 201
        types = len(set(lemma_list[n:]))
        ttr_200 = round((types / 200)*100, 3)
        ttr_200_all.append(ttr_200)
      count = count + 1
    ttr = round(sum(ttr_200_all)/len(ttr_200_all), 3)

  return ttr

#3. Вспомогательные подсчеты

##3.1. Поиск слов в квадратных скобках
Найти и проанализировать слова в квадратных скобках понадобилось, чтобы затем их корректно почистить внутри функции final_clean_the_text.

Для этого все судебные решения были сначала для удобства собраны в один файл, а затем в этом файле с помощью регулярных выражений были отобраны все слова в квадратных скобках. При этом чтобы сосредоточиться в рамках функции final_clean_the_text только на более частотных словах, они были подсчитаны в Counter и визуализированы в дата-фрейме по убыванию. В данном случае было решено использовать абсолютные частоты в связи с тем, что они использовались лишь в качестве общего ориентира при чистке слов, которые затем отбирались вручную.

In [ ]:
file = open('/content/Корпус одним файлом.txt', 'a', encoding='utf-8')
for i in os.listdir():
  if i.startswith('ECLI'):
    with open(f'/content/{i}', 'r', encoding='utf-8') as file_by_one:
      decision = file_by_one.read()
      decision_clean = initial_clean_the_text(decision)
      file.write(decision_clean)
file.close()

In [ ]:
with open('/content/Корпус одним файлом.txt', encoding = 'utf-8') as c:
  all_decisions = c.read()
words_in_brackets = re.findall(r'\[[\w\s/]+\]', all_decisions)
words_in_brackets_2 = [i.lower() for i in words_in_brackets]

data_frame_wib = pd.DataFrame.from_dict(Counter(words_in_brackets_2), orient='index', columns=['count'])
data_frame_wib_sorted = data_frame_wib['count'].sort_values(ascending=False)
data_frame_wib_sorted.to_excel('counter_brackets_all.xlsx')
data_frame_wib_sorted

## 3.2. Соотнесение длины текстов судебных решений с количеством лемм в них
Такое соотнесение понадобилось, поскольку в чат-боте для выбора пользователем размера судебного решения используется подсчет длины текста через количество наивных токенов, а для корректного подсчета TTR, в свою очередь, используется количество лемм.

Для наглядности результаты соотнесения были представлены в виде дата-фрейма. На его основе было выяснено, что за основу для окна в TTR можно взять 200 лемм, что соответствует примерно 600 наивных токенов в судебном решении.

In [ ]:
ENCLI_decision = []
number_lemmas = []
length_d = []
for i in os.listdir():
  if i.startswith('ECLI'):
    with open(f'/content/{i}', 'r', encoding='utf-8') as file_by_one:
      doc_one = file_by_one.read()
    frame = get_data_frame_common(final_clean_the_text(initial_clean_the_text(doc_one)))

    number_one_dec = frame.shape[0]
    number_lemmas.append(number_one_dec)

    length_one_dec = len(doc_one.split())
    length_d.append(length_one_dec)

    ENCLI_decision.append(i)

In [ ]:
df_number_lemmas = pd.DataFrame({'ECLI': ENCLI_decision, 'number_lemmas':  number_lemmas, 'length': length_d})
df_sorted = df_number_lemmas.sort_values(by='length', ascending=False)
df_sorted.to_excel('lemmas_sorted.xlsx')
df_sorted

#4. Создание csv-файла для базы данных чат-бота

##4.1. Формирование дата-фрейма (без ключевых слов)

В данной части кода формируется базовый дата-фрейм, к которому в последующем будет присоединена колонка с ключевыми словами.

В дата-фрейме хранится следующая информация:  
- название файла с судебным решением, в котором содержится его ECLI-код (код в базе данных судебных решений Нидерландов на сайте Rechtspraak.nl);
- порядковый номер судебного решения в дата-фрейме;
- длина судебного решения, подсчитанная через количество его наивных токенов;  
- TTR судебного решения, полученный в результате применения функции get_ttr;  
- текст судебного решения, почищенный с помощью функции initial_clean_the_text.

Также одновременно с формированием дата-фрейма собирается необходимая информация для дальнейшего подсчета TF-IDF - в list_of_texts_lemmatized собираются лемматизированные тексты судебных решений.

In [ ]:
dic = {}
list_of_texts_lemmatized = []

count = 0

for i in os.listdir():
  if i.startswith('ECLI'):
    with open(f'/content/{i}', 'r', encoding='utf-8') as file_by_one:
      decision = file_by_one.read()

    length_decision = len(decision.split())

    data_frame_decision = get_data_frame_common(final_clean_the_text(initial_clean_the_text(decision)))

    ttr_decision = get_ttr(data_frame_decision)

    data_frame_decision_key = get_data_frame_key_words(data_frame_decision)

    lemmas_list = data_frame_decision_key['lemma'].tolist()
    to_add = ' '.join(lemmas_list)
    list_of_texts_lemmatized.append(to_add)

    dic[i] = [count, length_decision, ttr_decision, initial_clean_the_text(decision)]

    count = count + 1
    print(f'Added! {count}')

In [ ]:
df_corpus = pd.DataFrame.from_dict(dic).T.rename(columns={0:'id_text', 1:'length', 2:'ttr', 3:'text'}).reset_index(names=['ECLI_code'])
df_corpus.to_csv('df_corpus_step_1.csv')
df_corpus.head()

##4.2. Добавление ключевых слов в дата-фрейм
Список лемматизированных текстов направляется в TfidfVectorizer для подсчета ключевых слов.

Для каждого текста отбираются 10 ключевых слов. Они сохраняются в список вместе с порядковым номером судебного решения и далее присоединяются к основному дата-фрейму по данному порядковому номеру.

После этого формируется последняя колонка дата-фрейма - в нем содержатся вместе ключевые слова и тексты судебных решений. Отсюда чат-бот будет получать текст судебного решения с ключевыми словами, который нужно будет направить пользователю.

In [ ]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(list_of_texts_lemmatized)
feature_names = tfidf_vectorizer.get_feature_names_out()

In [ ]:
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
df_tfidf.to_excel('tfidf corpus full.xlsx')
df_tfidf

In [ ]:
list_for_df_to_join = []
for i in range(df_tfidf.shape[0]):
  doc_scores = df_tfidf.iloc[i]
  sorted_scores_doc = doc_scores.sort_values(ascending=False)
  df_top = sorted_scores_doc.head(10)
  top_list_words = df_top.index.tolist()
  list_for_df_to_join.append([i, ', '.join(top_list_words)])
list_for_df_to_join[0]

In [ ]:
df_to_join = pd.DataFrame(list_for_df_to_join).rename(columns={0:'id_text', 1: 'key_words'})
df_to_join.to_excel('df_to_join.xlsx')
df_to_join

In [ ]:
df_corpus_full = pd.merge(df_corpus, df_to_join, on=['id_text'], how='outer')
df_corpus_full['key_words'] = df_corpus_full['key_words'].apply(lambda x: re.sub('_', '', x))
df_corpus_full['text_key_words'] = df_corpus_full['key_words'] + '\n' + df_corpus_full['text']
df_corpus_full.to_csv("df_corpus_full_key_ttr.csv", index=False)
df_corpus_full.head()